#### Load clsses/functions

In [1]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import trimesh
from skimage import measure
import meshplot as mp
from torch.utils.data import DataLoader, Dataset
import os
import time
from datetime import timedelta, datetime
import random
import math
from itertools import chain as chain

In [2]:
class ChairDataset(Dataset):
    def __init__(self, file_paths, n_points_to_load):
        self.file_paths = file_paths
        self.n_points_per_shape = 50000
        
        for file_path in file_paths:
            training_set = np.load(file_path)
            assert len(training_set['points']) == self.n_points_per_shape, f"{self.n_points_per_shape} data points expected, got: {training_set['points']}"
                
        self.n_points_to_load = n_points_to_load # number of points to load at once 
        
    def __getitem__(self, shape_idx):        
        training_set = np.load(self.file_paths[shape_idx]) # TODO: try mmap_mode='r'
        points = training_set['points']
        sdfs = training_set['sdf']
        
        n_shape_idx = np.full((self.n_points_to_load, 1), shape_idx, dtype=int)
        
        n_rand = random.sample(range(self.n_points_per_shape), self.n_points_to_load) # randomly pick 'n_points_to_load number' of indices  
        
        n_points = points[n_rand]
        
        n_sdf = sdfs[n_rand]
        n_sdf = np.resize(n_sdf, (self.n_points_to_load, 1))
        
        return n_shape_idx, n_points, n_sdf
    
    def __len__(self):
        return len(self.file_paths)


In [3]:
def load_files(all_file_or_not, n_files = 0):
    file_paths = []
    main_dir = '../data/03001627_sdfs/'

    if all_file_or_not: # loading all files
        n_files = 0
        for sub_dir in os.scandir(main_dir):
            if sub_dir.is_dir():
                for file in os.listdir(main_dir + sub_dir.name):
                    file_paths.append(main_dir + sub_dir.name + '/' + file) if file.endswith("sdf_samples.npz") else None
            n_files += 1
            
    else: # loading specific # of files
        for sub_dir in os.scandir(main_dir):
            if sub_dir.is_dir():
                for file in os.listdir(main_dir + sub_dir.name):
                    file_paths.append(main_dir + sub_dir.name + '/' + file) if file.endswith("sdf_samples.npz") else None
            if len(file_paths) == n_files:
                break
    
    print(f'total # of files: {n_files}')
    return file_paths


In [4]:
class MLP(nn.Module):
    def __init__(self, n_shapes, shape_code_length, n_inner_nodes):
        super(MLP, self).__init__()
        self.shape_code_length = shape_code_length
        self.shape_codes = nn.Embedding(n_shapes, shape_code_length) # shape code as an embedding
        nn.init.normal_(self.shape_codes.weight, mean=0, std=0.01)
        
        self.linear1 = nn.Linear(3 + shape_code_length, n_inner_nodes) # (x, y, z) + shape code 
        self.linear2 = nn.Linear(n_inner_nodes, n_inner_nodes)
        self.linear3 = nn.Linear(n_inner_nodes, n_inner_nodes)
        self.linear4 = nn.Linear(n_inner_nodes, n_inner_nodes - (3 + shape_code_length))
        self.linear5 = nn.Linear(n_inner_nodes, n_inner_nodes)
        self.linear6 = nn.Linear(n_inner_nodes, n_inner_nodes)
        self.linear7 = nn.Linear(n_inner_nodes, n_inner_nodes)
        self.linear8 = nn.Linear(n_inner_nodes, 1) # output a SDF value
        
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        
    def forward(self, shape_idx, x):
        shape_code = self.shape_codes(shape_idx.view(1, -1))
        shape_code = shape_code.view(-1, self.shape_code_length)
        shape_code_with_xyz = torch.cat((x, shape_code), dim=1) # concatenate horizontally
        
        out = self.linear1(shape_code_with_xyz)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.relu(out)
        out = self.linear3(out)
        out = self.relu(out)
        out = self.linear4(out)
        out = self.relu(out)
        out = self.linear5(torch.cat((out, shape_code_with_xyz), dim=1)) # skip connection
        out = self.relu(out)
        out = self.linear6(out)
        out = self.relu(out)
        out = self.linear7(out)
        out = self.relu(out)
        out = self.linear8(out)

        return out

    def add_noise_to_shape_codes(self, beta):
        self.shape_codes.weight.data += beta*(torch.rand_like(self.shape_codes.weight) * torch.std(self.shape_codes.weight, unbiased=False))


In [5]:
def generate_validation_points(file_paths, n_points_per_shape, n_points_to_generate):
    shape_idx = random.sample(range(len(file_paths)), 1)[0] # pick a shape randomly out of 7000 shapes
    n_shape_idx = np.full((n_points_to_generate, 1), shape_idx, dtype=int)
    
    training_set = np.load(file_paths[shape_idx]) 
    points = training_set['points']
    sdfs = training_set['sdf']

    rand = random.sample(range(n_points_per_shape), n_points_to_generate) # pick 1000 points randomly

    return torch.from_numpy(n_shape_idx), torch.from_numpy(points[rand]), torch.from_numpy(sdfs[rand])
    

In [6]:
file_paths = load_files(True)

n_points_per_shape = 50000
n_points_to_load = 2048  # n_points_to_load= n points loaded at once from a single file
batch_size = 10 # batch_size = n shapes loaded in one batch, not n data points
dataset = ChairDataset(file_paths=file_paths, n_points_to_load=n_points_to_load)
dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True) 


total # of files: 6778


#### Plotting inferred shapes

In [ ]:
# filename = './models/autodecoder_08052022_073446' # loading one point at a time
# model = MLP_old(10, 256, 256)
# filename = './models/autodecoder_08122022_202902' # loading multiple points at a time, 8 layers
# model = MLP(10, 256, 512)
# filename = './models/autodecoder_08122022_234636' # loading multiple points at a time, 3 layers, extra permutation
# model = MLP_old(10, 256, 256)
# filename = './models/autodecoder_08132022_003242' # loading multiple points at a time, 4 layers, extra permutation, 
# model = MLP_old(10, 256, 256)
# filename = './models/autodecoder_08132022_011401' # loading multiple points at a time, 4 layers, extra permutation, rigor=5 
# model = MLP_old(10, 256, 256)
# filename = './models/autodecoder_08132022_041952' # loading multiple points at a time, 8 layers, 16 shapes, extra permutation, rigor=5 
# model = MLP(16, 256, 512)
# filename = './models/autodecoder_08132022_170026' # loading multiple points at a time, 8 layers, 16 shapes, extra permutation, rigor=2, w.o clamping 
# model = MLP(16, 256, 512)
# filename = './models/autodecoder_08132022_190738' # loading 1024 points at a time, 8 layers, 6778 shapes, extra permutation, up to epoch 1813, without clamping
# model = MLP(6778, 256, 512)
# filename = './models/autodecoder_08142022_121303 # loading 1024 points at a time, 8 layers, 6778 shapes, extra permutation, up to epoch 1813, clamping + L1loss
# model = MLP(6778, 256, 512)
# 08142022_231250 # tanh, training loss increased
# model = MLP(6778, 256, 512)
# 08182022_105030 # remove max_norm, initialization only, regularization param: sigma = 1e2
# model = MLP(6778, 256, 512)
# 08182022_105028 # remove max_norm, initialization only, regularization param: sigma = 1e-2
# model = MLP(6778, 256, 512)
# 08182022_172451 # rerun on 10 shapes with the most updated network
# model = MLP(10, 256, 256)


In [8]:
x = np.linspace(-1, 1, 100, dtype=np.float32)
y = np.linspace(-1, 1, 100, dtype=np.float32)
z = np.linspace(-1, 1, 100, dtype=np.float32)
P = np.vstack(np.meshgrid(x,y,z)).reshape(3,-1).T  # format: [[x1, y1, z1], [x1, y1, z2], [] ...]
P = torch.from_numpy(P)


In [ ]:
# filename = './models/autodecoder_08182022_200131_28'
# model = MLP(6778, 256, 512)
# model.load_state_dict(torch.load(filename))
# model.eval()

# print(model.shape_codes.weight[0][:5])
# print(torch.std(model.shape_codes.weight))
# model.shape_codes.weight.data=0.1*torch.rand_like(model.shape_codes.weight) * torch.std(model.shape_codes.weight)
# print(model.shape_codes.weight[0][:5])
                                  
                                                           

In [9]:
# filename = './models/autodecoder_08182022_200131_86'
filename = './models/autodecoder_08182022_200131_172'


model = MLP(6778, 256, 512)
model.load_state_dict(torch.load(filename))
model.eval()

shape_idx = 1
shape_idx_tensor = torch.ones((P.shape[0], 1), dtype=torch.int) * shape_idx

volume = model(shape_idx_tensor,P).view(len(x), len(y), len(z)) 
volume = volume.detach().numpy()
verts, faces, normals, values = measure.marching_cubes(volume, 0)
mp.plot(verts, faces)


shape_idx = 100
shape_idx_tensor = torch.ones((P.shape[0], 1), dtype=torch.int) * shape_idx

volume = model(shape_idx_tensor,P).view(len(x), len(y), len(z)) 
volume = volume.detach().numpy()
verts, faces, normals, values = measure.marching_cubes(volume, 0)
mp.plot(verts, faces)

shape_idx = 300
shape_idx_tensor = torch.ones((P.shape[0], 1), dtype=torch.int) * shape_idx

volume = model(shape_idx_tensor,P).view(len(x), len(y), len(z)) 
volume = volume.detach().numpy()
verts, faces, normals, values = measure.marching_cubes(volume, 0)
mp.plot(verts, faces)

shape_idx = 600
shape_idx_tensor = torch.ones((P.shape[0], 1), dtype=torch.int) * shape_idx

volume = model(shape_idx_tensor,P).view(len(x), len(y), len(z)) 
volume = volume.detach().numpy()
verts, faces, normals, values = measure.marching_cubes(volume, 0)
mp.plot(verts, faces)

/home/paulaugguerrero_gmail_com/miniconda3/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(48.013362…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(52.002223…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(47.912233…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(47.499584…

#### plotting ground truth shapes

In [10]:
shape_idx = 1
mesh = trimesh.load(file_paths[shape_idx].split('sdf_samples.npz')[0] + 'mesh.obj')
mp.plot(mesh.vertices, mesh.faces, c=np.array([0, 0.9, 0.9]))

shape_idx = 100
mesh = trimesh.load(file_paths[shape_idx].split('sdf_samples.npz')[0] + 'mesh.obj')
mp.plot(mesh.vertices, mesh.faces, c=np.array([0, 0.9, 0.9]))

shape_idx = 300
mesh = trimesh.load(file_paths[shape_idx].split('sdf_samples.npz')[0] + 'mesh.obj')
mp.plot(mesh.vertices, mesh.faces, c=np.array([0, 0.9, 0.9]))

shape_idx = 600
mesh = trimesh.load(file_paths[shape_idx].split('sdf_samples.npz')[0] + 'mesh.obj')
mp.plot(mesh.vertices, mesh.faces, c=np.array([0, 0.9, 0.9]))


/home/paulaugguerrero_gmail_com/miniconda3/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(
/home/paulaugguerrero_gmail_com/miniconda3/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "uint32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [ ]:
# for shape_idx in range(10):
#     mesh = trimesh.load(file_paths[shape_idx].split('sdf_samples.npz')[0] + 'mesh.obj')
#     mp.plot(mesh.vertices, mesh.faces, c=np.array([0, 0.9, 0.9]))